#### 動かすにはipykernelのインストールが必要です
#### アプリ自体には不要なものであるので、requirements.txtへは入れていません

In [1]:
# connect.pyの中身
# uname() error回避
import platform
print(platform.uname())
# 意図は理解しきれていないが入れておく

import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# 環境変数のロード
load_dotenv()
AZURE_MY_SERVER = os.getenv("AZURE_MY_SERVER")
AZURE_MY_ADMIN = os.getenv("AZURE_MY_ADMIN")
AZURE_MY_PASSWORD = os.getenv("AZURE_MY_PASSWORD")
AZURE_MY_DATABASE = os.getenv("AZURE_MY_DATABASE")

# SSL証明書のパスを設定(connect.pyと同じ場所におく)
# base_path = os.path.dirname(os.path.abspath(__file__))
# ssl_cert_path = os.path.join(base_path, '../backend_env/DigiCertGlobalRootCA.crt.pem')
ssl_cert_path = 'backend_env/DigiCertGlobalRootCA.crt.pem'

# SQLAlchemyの接続URLを作成
connection_url = f"mysql+pymysql://{AZURE_MY_ADMIN}:{AZURE_MY_PASSWORD}@{AZURE_MY_SERVER}.mysql.database.azure.com/{AZURE_MY_DATABASE}?charset=utf8"

# SQLAlchemyエンジンを作成
engine = create_engine(connection_url, connect_args={"ssl": {"ca": ssl_cert_path}}, echo=True)

# セッションの作成
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# データベースセッションを取得するための関数
db = SessionLocal()




uname_result(system='Windows', node='DESKTOP-G50I8H1', release='10', version='10.0.22631', machine='AMD64')


C:\Users\Nishi-Hiro\AppData\Local\Temp\ipykernel_25448\3166847921.py:33: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
from fastapi import FastAPI, Depends, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy.orm import Session
from db_control import crud, connect, schemas
import base64
from typing import List



uname_result(system='Windows', node='DESKTOP-G50I8H1', release='10', version='10.0.22631', machine='AMD64')
2024-08-04 19:52:59,107 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-08-04 19:52:59,107 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-08-04 19:52:59,153 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-08-04 19:52:59,153 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-08-04 19:52:59,176 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-08-04 19:52:59,176 INFO sqlalchemy.engine.Engine [raw sql] {}
Connection established


In [3]:
def get_ec_sets_by_category(db: Session, category: str):
    return (
        db.query(
            EC_Set.ec_set_id,
            EC_Set.set_name,
            EC_Set.set_description,
        )
        .filter(EC_Set.category == category)
        .all()
    )

def get_ec_sets(category: str, db: Session):
    ec_sets = crud.get_ec_sets_by_category(db, category)

    return ec_sets

In [4]:
ec_sets = get_ec_sets(category="national", db = db)
display(ec_sets)

2024-08-04 19:53:06,042 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-08-04 19:53:06,047 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-08-04 19:53:06,091 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-08-04 19:53:06,093 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-08-04 19:53:06,114 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-08-04 19:53:06,116 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-08-04 19:53:06,165 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-04 19:53:06,217 INFO sqlalchemy.engine.Engine SELECT ec_sets.ec_set_id AS ec_sets_ec_set_id, ec_sets.set_name AS ec_sets_set_name, ec_sets.set_description AS ec_sets_set_description 
FROM ec_sets 
WHERE ec_sets.category = %(category_1)s
2024-08-04 19:53:06,217 INFO sqlalchemy.engine.Engine [generated in 0.00085s] {'category_1': 'national'}


[(1, '人気のナショナルブランド', '国民に愛される人気ブランドで、日々をもっと彩り豊かに！'),
 (2, '好みのナショナルブランド', 'あなたのスタイルにマッチする、お気に入りのナショナルブランド。'),
 (3, '意外なナショナルブランド', '新たな発見を楽しむ、意外なナショナルブランドセレクション。'),
 (4, 'xxx', 'xxx'),
 (5, 'xxx', 'xxx'),
 (6, 'xxx', 'xxx'),
 (7, 'xxx', 'xxx'),
 (8, 'xxx', 'xxx'),
 (9, 'xxx', 'xxx'),
 (10, 'xxx', 'xxx')]

In [5]:
from db_control import mymodels
import pandas as pd
from sqlalchemy import select, and_

In [6]:
# 1. ageとgenderの条件に当てはまるデータを取得
def get_filtered_data_by_age_gender(age, gender, category):
    query = (
        select(mymodels.Survey)
        .join(mymodels.Brand)
        .where(
            and_(
                mymodels.Survey.age_lower_limit < age,
                mymodels.Survey.age_upper_limit > age,
                mymodels.Survey.gender == gender,
                mymodels.Brand.category == category,
            )
        )
    )

    results = db.execute(query).mappings().all()
    
    return results

# 2. 抽出したデータに含まれるbrand_idの種類を取得
def get_unique_brand_ids(results):
    brand_ids = {row['Survey'].brand_id for row in results}
    return brand_ids

# 3. 各brand_idについて、get_filtered_dataを使用してデータを取得し、一つのDataFrameを作成
def get_combined_data(age, gender, category):
    results = get_filtered_data_by_age_gender(age, gender, category)
    brand_ids = get_unique_brand_ids(results)
    
    combined_data = pd.DataFrame()
    
    for brand_id in brand_ids:
        df = get_filtered_data(brand_id, age, gender)
        combined_data = pd.concat([combined_data, df], ignore_index=True)
    
    return combined_data

In [7]:
# データ抽出関数
def get_filtered_data(brand_id, age, gender):
    query = (
        select(mymodels.Survey)
        .where(
            and_(
                mymodels.Survey.brand_id == brand_id,
                mymodels.Survey.age_lower_limit < age,
                mymodels.Survey.age_upper_limit > age,
                mymodels.Survey.item_id.in_([1, 2, 3, 4, 5, 6, 7, 8]),
                mymodels.Survey.gender == gender
            )
        )
    )

    # 出力結果をうまく扱えなかったので辞書形式にする
    results = db.execute(query).mappings().fetchall()
    
    # 結果をデバッグ用に表示
    # for r in results:
    #     survey = r['Survey']  # 辞書からオブジェクトを取り出す
    #     print(f"survey_id: {survey.survey_id}, item_id: {survey.item_id}, brand_id: {survey.brand_id}, score: {survey.score}, age_lower_limit: {survey.age_lower_limit}, age_upper_limit: {survey.age_upper_limit}, gender: {survey.gender}, response_count: {survey.response_count}")
    
    # スコアを8次元のベクトルとして整理
    data = {'brand_id': [brand_id], 'age': [age], 'gender': [gender], 'id1': [None], 'id2': [None], 'id3': [None], 'id4': [None], 'id5': [None], 'id6': [None], 'id7': [None], 'id8': [None]}
    
    for row in results:
        survey = row['Survey']  # 辞書からオブジェクトを取り出す
        item_id = survey.item_id
        if 1 <= item_id <= 8:
            data[f'id{item_id}'][0] = survey.score
    
    df = pd.DataFrame(data)
    return df

In [8]:
# 例としてageが35でgenderが1の場合のデータを取得し、統合
age = 35
gender = 1
category = "craft"
combined_df = get_combined_data(age, gender, category)
print(combined_df)

2024-08-04 19:53:22,529 INFO sqlalchemy.engine.Engine SELECT surveys.survey_id, surveys.item_id, surveys.brand_id, surveys.score, surveys.age_lower_limit, surveys.age_upper_limit, surveys.gender, surveys.response_count 
FROM surveys INNER JOIN brands ON brands.brand_id = surveys.brand_id 
WHERE surveys.age_lower_limit < %(age_lower_limit_1)s AND surveys.age_upper_limit > %(age_upper_limit_1)s AND surveys.gender = %(gender_1)s AND brands.category = %(category_1)s
2024-08-04 19:53:22,530 INFO sqlalchemy.engine.Engine [generated in 0.00127s] {'age_lower_limit_1': 35, 'age_upper_limit_1': 35, 'gender_1': 1, 'category_1': 'craft'}
2024-08-04 19:53:22,619 INFO sqlalchemy.engine.Engine SELECT surveys.survey_id, surveys.item_id, surveys.brand_id, surveys.score, surveys.age_lower_limit, surveys.age_upper_limit, surveys.gender, surveys.response_count 
FROM surveys 
WHERE surveys.brand_id = %(brand_id_1)s AND surveys.age_lower_limit < %(age_lower_limit_1)s AND surveys.age_upper_limit > %(age_uppe

In [9]:
# 1. 指定したuser_idに関する情報を抽出
def get_user_preferences(user_id):
    query = (
        select(mymodels.Preference)
        .where(mymodels.Preference.user_id == user_id)
    )

    results = db.execute(query).mappings().fetchall()
    
    return results


# 2. item_idに対応するscoreを8次元のベクトルとして整理し、3. 得られた結果をDataFrame形式で出力
def get_user_preference_vector(user_id):
    results = get_user_preferences(user_id)
    
    # スコアを8次元のベクトルとして整理
    data = {'user_id': [user_id], 'id1': [None], 'id2': [None], 'id3': [None], 'id4': [None], 'id5': [None], 'id6': [None], 'id7': [None], 'id8': [None]}
    
    for row in results:
        preference = row['Preference']  # 辞書からオブジェクトを取り出す
        item_id = preference.item_id
        if 1 <= item_id <= 8:
            data[f'id{item_id}'][0] = preference.score
    
    df = pd.DataFrame(data)
    return df

In [25]:
user_id = 1  # 本来はJWTから取得する
user_id = 1
user_df = get_user_preference_vector(user_id)
print(user_df)

2024-08-04 20:09:49,468 INFO sqlalchemy.engine.Engine SELECT preferences.user_id, preferences.item_id, preferences.score 
FROM preferences 
WHERE preferences.user_id = %(user_id_1)s
2024-08-04 20:09:49,469 INFO sqlalchemy.engine.Engine [cached since 975.4s ago] {'user_id_1': 1}
   user_id  id1  id2  id3  id4  id5  id6  id7  id8
0        1  3.0  2.0  2.0  1.0  2.0  1.0  4.0  3.0


In [11]:
from scipy.spatial.distance import cosine

In [18]:
# Cos類似度を計算する関数
def calculate_cosine_similarity(vector1, vector2):
    return 1 - cosine(vector1, vector2)

# combined_dfの8次元ベクトルとuser_dfの8次元ベクトルでcos類似度を計算し、rec_scoreに格納する関数
def add_recommendation_scores(combined_df, user_vector):
    combined_df['rec_score'] = 0.0
    user_vector = user_vector.iloc[0, 1:].values.tolist()  # user_vectorの8次元ベクトルを取得
    print(user_vector)
    
    for idx, row in combined_df.iterrows():
        combined_vector = row[['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8']].values.tolist()
        rec_score = calculate_cosine_similarity(combined_vector, user_vector)
        combined_df.at[idx, 'rec_score'] = rec_score
        
    # 'rec_score'を降順でソート
    combined_df = combined_df.sort_values(by='rec_score', ascending=False).reset_index(drop=True)
    
    return combined_df

In [23]:
user_df = pd.DataFrame({
    'user_id': [1],
    'id1': [4.0/2],
    'id2': [4.0/2],
    'id3': [3.0/2],
    'id4': [3.0/2],
    'id5': [5.0/2],
    'id6': [2.0/2],
    'id7': [5.0/2],
    'id8': [3.0/2]
})

In [26]:
combined_df_with_scores = add_recommendation_scores(combined_df, user_df)
print(combined_df_with_scores)

[3.0, 2.0, 2.0, 1.0, 2.0, 1.0, 4.0, 3.0]
    brand_id  age  gender      id1      id2      id3      id4      id5  \
0         19   35       1  4.00000  4.00000  3.00000  3.00000  5.00000   
1         24   35       1  3.00000  3.00000  2.00000  1.00000  4.00000   
2         23   35       1  2.00000  1.00000  1.00000  1.00000  2.00000   
3         11   35       1  3.50000  2.00000  4.00000  4.00000  4.50000   
4         25   35       1  2.00000  3.00000  3.00000  3.00000  2.00000   
5         15   35       1  3.33333  2.66667  3.00000  3.00000  2.66667   
6         20   35       1  4.00000  5.00000  4.00000  2.00000  3.00000   
7         29   35       1  3.00000  2.00000  4.00000  3.00000  1.00000   
8         18   35       1  3.00000  4.00000  4.50000  2.50000  2.00000   
9         13   35       1  1.66667  2.33333  3.33333  3.33333  1.00000   
10        14   35       1  3.00000  4.66667  4.33333  3.66667  3.66667   
11        22   35       1  1.00000  3.00000  3.00000  2.00000  2.00000 

In [31]:
combined_df_with_scores.head(3)[["brand_id"]]

,brand_id
0,15
1,23
2,29


In [27]:
import numpy as np

In [28]:
# ユークリッド距離で評価する

# ユークリッド距離を計算する関数
def calculate_euclidean_distance(vector1, vector2):
    return np.linalg.norm(np.array(vector1) - np.array(vector2))

# combined_dfの8次元ベクトルとuser_dfの8次元ベクトルでユークリッド距離を計算し、distanceに格納する関数
def add_recommendation_scores(combined_df, user_vector):
    combined_df['distance'] = 0.0
    user_vector = user_vector.iloc[0, 1:].values.tolist()  # user_vectorの8次元ベクトルを取得
    
    for idx, row in combined_df.iterrows():
        combined_vector = row[['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8']].values.tolist()
        distance = calculate_euclidean_distance(combined_vector, user_vector)
        combined_df.at[idx, 'distance'] = distance
    
    # 'distance'を昇順でソート
    combined_df = combined_df.sort_values(by='distance', ascending=True).reset_index(drop=True)
    return combined_df

In [29]:
combined_df_with_scores = add_recommendation_scores(combined_df, user_df)
print(combined_df_with_scores)

    brand_id  age  gender      id1      id2      id3      id4      id5  \
0         15   35       1  3.33333  2.66667  3.00000  3.00000  2.66667   
1         23   35       1  2.00000  1.00000  1.00000  1.00000  2.00000   
2         29   35       1  3.00000  2.00000  4.00000  3.00000  1.00000   
3         13   35       1  1.66667  2.33333  3.33333  3.33333  1.00000   
4         22   35       1  1.00000  3.00000  3.00000  2.00000  2.00000   
5         24   35       1  3.00000  3.00000  2.00000  1.00000  4.00000   
6         25   35       1  2.00000  3.00000  3.00000  3.00000  2.00000   
7         18   35       1  3.00000  4.00000  4.50000  2.50000  2.00000   
8         17   35       1  3.00000  5.00000  3.00000  3.00000  2.00000   
9         20   35       1  4.00000  5.00000  4.00000  2.00000  3.00000   
10        19   35       1  4.00000  4.00000  3.00000  3.00000  5.00000   
11        26   35       1  1.50000  1.25000  3.25000  3.50000  1.75000   
12        11   35       1  3.50000  2.

In [19]:
db.close

<bound method Session.close of <sqlalchemy.orm.session.Session object at 0x000001E2D1B31B10>>

In [ ]:
# 例としてbrand_idが1でageが25の場合のデータを取得
brand_id = 6
age = 35
gender = 1
df = get_filtered_data(brand_id, age, gender)
print(df)

2024-08-04 16:57:49,907 INFO sqlalchemy.engine.Engine SELECT surveys.survey_id, surveys.item_id, surveys.brand_id, surveys.score, surveys.age_lower_limit, surveys.age_upper_limit, surveys.gender, surveys.response_count 
FROM surveys 
WHERE surveys.brand_id = %(brand_id_1)s AND surveys.age_lower_limit < %(age_lower_limit_1)s AND surveys.age_upper_limit > %(age_upper_limit_1)s AND surveys.item_id IN (%(item_id_1_1)s, %(item_id_1_2)s, %(item_id_1_3)s, %(item_id_1_4)s, %(item_id_1_5)s, %(item_id_1_6)s, %(item_id_1_7)s, %(item_id_1_8)s) AND surveys.gender = %(gender_1)s
2024-08-04 16:57:49,908 INFO sqlalchemy.engine.Engine [cached since 738.8s ago] {'brand_id_1': 6, 'age_lower_limit_1': 35, 'age_upper_limit_1': 35, 'gender_1': 1, 'item_id_1_1': 1, 'item_id_1_2': 2, 'item_id_1_3': 3, 'item_id_1_4': 4, 'item_id_1_5': 5, 'item_id_1_6': 6, 'item_id_1_7': 7, 'item_id_1_8': 8}
   brand_id  age  gender   id1   id2   id3   id4   id5   id6   id7   id8
0         6   35       1  None  None  None  None